# CNN  digital recognition with tensorflow

###  **Please upvote if you like my approach or if you learned something from this notebook. Your support gives me motivation to create interesting stuff. Thank you.** 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# importing the libraries 

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau
from keras.utils.np_utils import to_categorical

from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, LeakyReLU
from keras.optimizers import RMSprop

<h1 style="color:black" align="center">Handwritten digits classification using neural network</h1>


In this notebook we will classify handwritten digits using a simple neural network which has only input and output layers. We will than add a hidden layer and see how the performance of the model improves

after reading the data we are going to normalize it by divid each pixel with 255 

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
train.head()
y = train.label
x = train.drop(columns= 'label')
x = x/255
x.head()

y = to_categorical(y, num_classes = 10)
print(y.shape)

here we are going to use sklearn models to fit the data into a NN model that have **64 nuroal** in the frist hidden layer, **32 nuroal **in the second hidden layer, and** 10 output** layers 

 <img src= "https://www.researchgate.net/profile/Woongje-Sung/publication/322313274/figure/fig1/AS:653009751134212@1532701065757/LeNet-59-CNN-for-handwritten-digit-recognition-task.png" alt ="nuroal network "  align="center" style='width: 500px;' >


In [ ]:
X_train,X_val,y_train,y_val = train_test_split(x,y,test_size=0.2, random_state=100)
print(X_train.shape ,X_val.shape, y_train.shape, y_val.shape)
X_train= X_train.values.reshape( -1 , 28 , 28 , 1)
X_val= X_val.values.reshape( -1 , 28 , 28 , 1)
print(X_train.shape ,X_val.shape)


# model 
>* first we are going to create CNN with 32 filters,  kernel 5x5 , and relu activation funciton  followed by Max pooling layer 
>* then create the second CNN layer with 64 filters,  kernel 3x3 , and relu activation funciton  followed by Max pooling layer 
>* then using flat the data into 1 dimension using flatten funciton, with 1024 NN,  relu activation function, and 50% dropout 
>* followed by second NN with 256 neuron with leaky Relu activation function, 50% dropout 
>* lastly the output NN with 10 neurons that calculated using softmax activation function. 

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='tanh', input_shape = (28,28,1)))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))


model.add(Flatten())
model.add(Dense(1024, activation=LeakyReLU(alpha=0.3)))
model.add(Dropout(0.5))
model.add(Dense(256, activation=LeakyReLU(alpha=0.2)) )
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# Compile the model
model.compile(optimizer = optimizer , loss = "binary_crossentropy", metrics=["accuracy"])
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
epochs = 25 
batch_size = 80

In [ ]:
model.summary()

# model fitting

In [ ]:
# Fit the model 
print(X_train.shape ,X_val.shape, y_train.shape, y_val.shape)


history = model.fit(x= X_train,y= y_train, batch_size=batch_size,
                              epochs = epochs, validation_data = (X_val, y_val),
                              verbose = 2, steps_per_epoch=(X_train.shape[0] // batch_size)
                            , callbacks=[learning_rate_reduction])

# model predicting

In [ ]:
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
print(test.shape)

xtest = test/255
xtest= xtest.values.reshape( -1 , 28 , 28 , 1)
print(xtest.shape)
ypre= model.predict(xtest)
ypre

In [ ]:
sample = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
sample1 = sample.drop(columns= 'ImageId')
sample1[:5]

# display some answers 

In [ ]:
y_predicted = [np.argmax(i) for i in ypre]
y_predicted[:5]

In [ ]:
print(len(sample1),'<-- sample prediction-->  ' ,  len(y_predicted))

In [ ]:
y_predicted = pd.DataFrame(y_predicted)
newY = pd.concat([sample.drop(columns='Label'), y_predicted], axis = 1)


In [ ]:
newY.columns=['ImageId', 'Label']
newY

# Evaluate the Modle 

In [ ]:
model.evaluate(xtest,ypre)

# saving the Model

In [ ]:
newY.to_csv('my_submission.csv', index= False)
